In [1]:
import os, sys, argparse

# sys.path.append(os.path.join(os.getcwd(), '.'))

# import additions.constants as consts
import numpy as np
import pandas as pd

In [2]:
def gen_mixed_dataset():
    # folder = os.path.join(consts.DIR_CSV, 'corr', 'mixed')
    # filename = os.path.join(folder, 'raw_data.pkl.gzip')
    # if not os.path.exists(folder):
    #     os.mkdir(folder)

    # 8 features, 4 num, 4 cat (n1, n2, n3, n4, c5, c6, c7, c8)
    # n0 € [0..1] Useless feature
    # n1 € [0..1]
    # n2 € 2 * n1 ± N(0, 0.2)
    # n3 € [0..1]
    # n4 € f(c8) -> [a: 0.1, b: 0.3, c: 0.5, d: 0.7, e: 0.9] ± N(0, 0.2)
    # c5 € [a, b, c, d, e]
    # c6 € upper(c5) with 20% chance of random change to another category
    # c7 € f(n3) -> {a:0..<0.2, b:0.2..<0.4, c:0.4..<0.6, d:0.6..<0.8, e:0.8..1.0} with 20% chance of random change to another category
    # c8 € [a, b, c, d, e]
    # c9 € [a, b, c, d, e] Useless feature

    d = 1000
    r = 0.2

    n0 = np.random.random((d, 1))
    n1 = np.random.random((d, 1))
    n2 = n1 * 2 + np.random.normal(0, r, (d, 1))
    n3 = np.random.random((d, 1))

    def ccol(ra=None):
        c = np.array([[""] * d]).T
        if ra is None:
            ra = np.random.random((d, 1))
        np.putmask(c, ra < 0.2, "a")
        np.putmask(c, (ra >= 0.2) & (ra < 0.4), "b")
        np.putmask(c, (ra >= 0.4) & (ra < 0.6), "c")
        np.putmask(c, (ra >= 0.6) & (ra < 0.8), "d")
        np.putmask(c, ra > 0.8, "e")
        return c

    c5 = ccol()
    c6 = c5.copy()
    np.putmask(c6, np.random.random((d, 1)) < 0.2, ccol())
    c6[:, 0] = np.apply_along_axis(lambda x: x[0].upper(), 1, c6)
    c7 = ccol(ra=n3)
    np.putmask(c7, np.random.random((d, 1)) < 0.2, ccol())
    c8 = ccol()
    c9 = ccol()

    n4 = np.zeros((d, 1))
    np.putmask(n4, c8 == "a", 0.1)
    np.putmask(n4, c8 == "b", 0.3)
    np.putmask(n4, c8 == "c", 0.5)
    np.putmask(n4, c8 == "d", 0.7)
    np.putmask(n4, c8 == "e", 0.9)
    n4 += np.random.normal(0, r, (d, 1))

    print(n0.shape, n1.shape, n2.shape, n3.shape, n4.shape)
    print(c5.shape, c6.shape, c7.shape, c8.shape, c9.shape)

    data = np.concatenate([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9], axis=1)
    cols = ["N0", "N1", "N2", "N3", "N4", "C5", "C6", "C7", "C8", "C9"]

    df = pd.DataFrame(data, columns=cols)
    df[cols[:5]] = df[cols[:5]].astype(float)
    # df.to_pickle(filename)

    return df


gen_mixed_dataset().head()

(1000, 1) (1000, 1) (1000, 1) (1000, 1) (1000, 1)
(1000, 1) (1000, 1) (1000, 1) (1000, 1) (1000, 1)


,N0,N1,N2,N3,N4,C5,C6,C7,C8,C9
0,0.505321,0.284864,0.655726,0.150357,0.395861,c,C,a,b,d
1,0.409260,0.031902,0.320296,0.653137,0.331237,b,B,d,b,a
2,0.904029,0.746586,1.328019,0.417862,0.838134,c,C,c,d,a
3,0.063662,0.621714,1.291321,0.526868,0.456212,e,E,c,c,d
4,0.740151,0.773944,1.454796,0.072190,0.444516,d,E,a,b,b


In [3]:
# ----------- Linear Drift -------------------------
def _lin_incr(begin, end, i_begin, i_end, chunk):
    """
    Linear Drift : Linearly increments the values

    Parameters:

    begin = 0
    end = 10
    i_begin = 2
    i_end = 5
    chunk = 10
    Calculation:

    vals = 5 - 2 + 1 = 4
    srange = 10 - 0 = 10
    step = 10 / 4 = 2.5
    moving_values = np.arange(0, 10, 2.5) = [0.0, 2.5, 5.0, 7.5]
    Concatenation:

    [0]*2 = [0, 0] (first 2 elements are 0)
    moving_values = [0.0, 2.5, 5.0, 7.5] (linearly spaced values)
    [10]*(10 - 5 - 1) = [10, 10, 10, 10] (remaining elements are 10)
    Result:

    The final array is [0, 0, 0.0, 2.5, 5.0, 7.5, 10, 10, 10, 10]
    """
    # number of values to generate, inclusive of the beginning index OR end index
    vals = i_end - i_begin + 1  # 5 - 2 + 1 = 4 -------- this is from moving values
    # range of values to generate from
    srange = end - begin  # generate values from a range of 0 to 10 ....(10-0)
    # from the range of the values, and the number of values to generate,
    # derive the step size, inorder to generate the no of values given in given range
    step = srange / vals  # ------------- this is for moving valuez
    # moving_values = np.arange(begin, end * (1 + 1 / (vals + 1)), step)
    moving_values = np.arange(begin, end, step)
    return np.append(
        np.append(
            [begin] * i_begin, moving_values
        ),  # combine begin value, "i_begin" number of times,
        # before even starting with, the values begin, then after that append the moving values
        [end]
        * (
            chunk - i_end - 1
        ),  # now add the end values till with remaining size indicated by chunk
    )
    # result : [0, 0, 0.0, 2.5, 5.0, 7.5, 10, 10, 10, 10]
    # gives a linear increment in values


_lin_incr(begin=0, end=10, i_begin=2, i_end=5, chunk=10)

array([ 0. ,  0. ,  0. ,  2.5,  5. ,  7.5, 10. , 10. , 10. , 10. ])

In [4]:
# ------------ Blend two concepts gradually over a specified range -----------------
def _gradual(concept1, concept2, i_begin, i_end, chunk):
    """
    The function _gradual is designed to blend two concepts (arrays of values) gradually over a specified range of indices.

    Parameters:
    concept1: The first concept, which can be a NumPy array, a pandas Series, or a single value.
    concept2: The second concept, which can also be a NumPy array, a pandas Series, or a single value.
    i_begin: The starting index of the range over which the blending occurs.
    i_end: The ending index of the range over which the blending occurs.
    chunk: The size of the arrays that will be generated if the concepts are not already arrays.
    """
    if not isinstance(concept1, np.ndarray):
        if isinstance(concept1, pd.Series):
            concept1 = concept1.to_numpy()
        else:
            concept1 = np.array([concept1] * chunk)
    if not isinstance(concept2, np.ndarray):
        if isinstance(concept1, pd.Series):
            concept2 = concept2.to_numpy()
        else:
            concept2 = np.array([concept2] * chunk)
    # The function _lin_incr (presumably defined elsewhere) generates an array called chances which linearly
    # increases from 0 to 1 over the specified range from i_begin to i_end, with a length of chunk.
    chances = _lin_incr(0, 1, i_begin, i_end, chunk)
    # Create a mask based on random values and the chance array:
    mask = np.random.random(chunk) < chances
    t = concept1.copy()
    # Changes elements of an array based on conditional and input values.
    # ------------ Blend two concepts gradually over a specified range -----------------
    # ------------ Blend two concepts gradually over a specified range -----------------
    # ------------ Blend two concepts gradually over a specified range -----------------
    # t = [1, 2, 3, 4], mask = [true, false, true , false], concept2 = [10, 8, 7, 6]
    # t= [1, 8, 3, 6]
    # ------------ Blend two concepts gradually over a specified range -----------------
    np.putmask(t, mask, concept2)
    return t

In [5]:
def change_label_func(filepath, class_col="class"):
    df = pd.read_pickle(filepath)

    def re_class(
        b: int,
        e: int,
        ccol: str,
        true_ncols: list,
        false_ncols: list,
        true_coefs: list,
        false_coefs: list,
        thrs: list,
        revs: list,
        flip_chance: float = 0.0,
        df: pd.DataFrame = df,
        lcol: str = class_col,
    ):
        """
        Parameters:

        b, e: Start and end indices of the rows to be processed.
        ccol: Column name or array indicating a specific condition (e.g., "c7").
        true_ncols, false_ncols: Column names used for the condition checks.
        true_coefs, false_coefs: Coefficients applied to the columns for the condition checks.
        thrs: Threshold values for the conditions.
        revs: Booleans to reverse the condition logic.
        flip_chance: Probability of flipping the label.
        df: DataFrame to be processed.
        lcol: The label column to be updated.
        """
        # b = 0, e = 999 => 1000
        t = np.zeros(e - b + 1).astype("bool")
        temp = np.zeros(e - b + 1)

        #  ccol: str = "c7",
        # true_ncols: list = ["n2", "n3"],
        # false_ncols: list = ["n3", "n4"],
        # true_coefs: list =  [1, 1]
        # false_coefs: list = [1, 1]
        # thres = [0.5, -0.5],
        # revs = [False, False],
        # flip chance = 1/32

        for ncol, coef in zip(true_ncols, true_coefs):
            temp += df.loc[b:e, ncol] * coef
        temp = (temp < thrs[0]) ^ revs[0]
        if isinstance(ccol, str):
            temp = (df.loc[b:e, ccol] == "b") & temp
        else:
            temp = (ccol == "b") & temp
        t = t | temp

        for ncol, coef in zip(false_ncols, false_coefs):
            temp += df.loc[b:e, ncol] * coef
        temp = (temp < thrs[1]) ^ revs[1]
        if isinstance(ccol, str):
            temp = (df.loc[b:e, ccol] != "b") & temp
        else:
            temp = (ccol != "b") & temp
        t = t | temp

        flip = np.random.random(e - b + 1) < flip_chance
        print(flip_chance, flip.sum())
        df.loc[b:e, lcol] = t ^ flip

    re_class(
        0,
        9999,
        "c7",
        ["n2", "n3"],
        ["n3", "n4"],
        [1, 1],
        [1, 1],
        [0.5, -0.5],
        [False, False],
        1 / 32,
    )
    re_class(
        10000,
        10999,
        "c7",
        ["n2", "n3"],
        ["n3", "n4"],
        [1, 1],
        [1, 1],
        [_lin_incr(0.5, -1, 0, 499, 1000), _lin_incr(-0.5, 0, 0, 499, 1000)],
        [False, False],
        1 / 32,
    )
    grad = _gradual(df.loc[11000:11999, "c7"], df.loc[11000:11999, "c6"], 0, 499, 1000)
    grad = pd.Series(grad, index=range(11000, 12000))
    re_class(
        11000,
        11999,
        grad,
        ["n2", "n3"],
        ["n3", "n4"],
        [1, 1],
        [1, 1],
        [-1, 0],
        [False, False],
        1 / 32,
    )
    re_class(
        12000,
        12999,
        "c5",
        ["n0", "n4"],
        ["n1"],
        [1, 1],
        [1, 1],
        [0.5, 0.5],
        [False, True],
        1 / 32,
    )

    df[class_col] = df[class_col].replace({0: "A", 1: "B"})

    return df

In [6]:
def change_label_func_splits(filepath, class_col="class"):
    df = pd.read_pickle(filepath)
    df[class_col] = df[class_col].replace({"A": 0, "B": 1})
    folder = os.path.join(os.path.dirname(filepath), "prepared")

    for i in range(30):
        for m in [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]:
            datapath = os.path.join(folder, str(i), str(m), "labeled_set.pkl.gzip")
            data = pd.read_pickle(datapath)
            data.loc[data.index, class_col] = df.loc[data.index, class_col]
            data.to_pickle(datapath)
            for num in range(0, 12950, 50):
                datapath = os.path.join(folder, str(i), str(m), f"data{num}.pkl.gzip")
                data = pd.read_pickle(datapath)
                data.loc[data.index, class_col] = df.loc[data.index, class_col]
                data.to_pickle(datapath)

In [12]:
def ncol(stddev=1, ra=None, ra2=None, mult_ra=1, mult_ra2=1):
    """
    The function ncol is designed to generate a sequence of numbers with an optional noise component.

    Parameters
    --------------------------------------------------------------

    stddev: default = 1
            This parameter controls the standard deviation of the noise to be added.
            A higher value results in more variability (noise) in the generated sequence.
            If stddev is set to 0, no noise is added.

    ra: default = None
        This is an optional parameter. default = None
        If provided, it is used as a base sequence to which the noise and other modifications are applied.

    ra2: default = None
         This is another optional parameter.
         If provided, it is used alongside ra to further modify the generated sequence.

    mult_ra: default = 1
             This multiplier is applied to the ra sequence if ra is provided.

    mult_ra2: default = 1
              This multiplier is applied to the ra2 sequence if ra2 is provided.
    """
    if stddev == 0:
        noise = np.zeros(
            chunk
        )  # If stddev is 0, noise is an array of zeros with length chunk.
    else:
        # Otherwise noise is generated as a sequence of random numbers from a normal distribution with the specified standard deviation stddev.
        noise = np.random.normal(scale=stddev, size=chunk)
    # else: noise = np.random.uniform(high=stddev, size=chunk)
    if ra is None:
        return noise
    elif ra2 is None:
        return mult_ra * (ra + noise)
    else:
        return (mult_ra * ra) + (mult_ra2 * ra2) + noise


chunk = 1000  # size of array to return
err = 0.125  # err is set to 0.125, which will be used as the standard deviation for noise in the second call to ncol.
n1 = (
    ncol()
)  # This generates n1 as a sequence of 1000 random numbers from a normal distribution with a standard deviation of 1.
# This generates n2 as a sequence where each element is computed as 2 × (𝑛1[𝑖] + noise[𝑖]),
# where noise is a new sequence of random numbers from a normal distribution with a standard deviation of 0.125.
n2 = ncol(err, n1, mult_ra=2)

In [27]:
def ccol(flip_chance=0, ra=None, ra2=None, ra_thr=0, reverse_cats=False):
    """
    The ccol function generates a binary array (with values True or False) based on various conditions and parameters.

    Parameters:
    -----------------------------------------------------------------------------
    flip_chance: Probability (between 0 and 1) of flipping each value in the array.
    ra: An optional array or value to determine the condition.
    ra2: A second optional array for logical operations.
    ra_thr: A threshold value for comparisons.
    reverse_cats: If True, reverses the values in the resulting array.
    """
    if ra is None:
        ra = np.random.random(chunk)
        c = ra < ra_thr
    elif ra2 is None:
        if np.issubdtype(ra.dtype, np.number):
            c = ra < ra_thr
        else:
            c = ra
    else:
        c = np.logical_xor(ra, ra2)

    flip = np.random.random(chunk) < flip_chance
    c = np.logical_xor(c, flip)  # true only when one of the value is TRUE
    if reverse_cats:
        c = ~c
    return c


dataset = np.zeros((0, 11))
chunk = 1000
t_end = chunk // 2 - 1
err = 0.125
c5 = ccol(ra_thr=0.5)
c6 = ccol(ra_thr=0.5)
c7 = ccol(err, c6)
# c8 = ccol(err, n3, ra_thr=0)
# c9 = ccol(err, c6, c8)

In [7]:
def cfpdss_dataset():
    folder = os.path.join("consts.DIR_CSV", "cfpdss")
    filename = os.path.join(folder, "raw_data.pkl.gzip")
    # 10 features, 1 label 2 classes
    # n0 € N1
    # n1 € N1
    # n2 € a * (n1 ± N.25)
    # n3 € N1
    # n4 € a * n1 + (1 - a) * n3
    # c5 € [a, b, c, d], no correlation
    # c6 € [a, b, c, d]
    # c7 € c5 with 25% chance to reroll random category
    # c8 € f(n3) -> {a:0..<0.25, b:0.25..<0.5, c:0.5..<0.75, d:0.75..<1.0} with 25% chance to reroll random category
    # c9 € [a, b, c, d]
    # l € {0, 1} c7 n2

    dataset = np.zeros((0, 11))
    chunk = 1000
    t_end = chunk // 2 - 1
    err = 0.125

    def lin_incr(begin, end, i_begin=0, i_end=t_end):
        return _lin_incr(begin, end, i_begin, i_end, chunk)

    def gradual(concept1, concept2, i_begin=0, i_end=t_end):
        return _gradual(concept1, concept2, i_begin, i_end, chunk)

    def tfunc(
        ccol1,
        ccol2,
        ncols1,
        ncols2,
        ncols3,
        ncols4,
        coefs1,
        coefs2,
        coefs3,
        coefs4,
        thrs,
        revs,
        flip_chance=0,
    ):
        t = np.zeros(chunk).astype("bool")

        temp = np.zeros(chunk)
        for ncol, coef in zip(ncols1, coefs1):
            temp += ncol * coef
        temp = ccol1 & ccol2 & ((temp < thrs[0]) ^ revs[0])
        t = t | temp

        temp = np.zeros(chunk)
        for ncol, coef in zip(ncols2, coefs2):
            temp += ncol * coef
        temp = ccol1 & (~ccol2) & ((temp < thrs[1]) ^ revs[1])
        t = t | temp

        temp = np.zeros(chunk)
        for ncol, coef in zip(ncols3, coefs3):
            temp += ncol * coef
        temp = (~ccol1) & ccol2 & ((temp < thrs[2]) ^ revs[2])
        t = t | temp

        temp = np.zeros(chunk)
        for ncol, coef in zip(ncols4, coefs4):
            temp += ncol * coef
        temp = (~ccol1) & (~ccol2) & ((temp < thrs[3]) ^ revs[3])
        t = t | temp

        flip = np.random.random(chunk) < flip_chance
        return np.logical_xor(t, flip)

    def ncol(stddev=1, ra=None, ra2=None, mult_ra=1, mult_ra2=1):
        if stddev == 0:
            noise = np.zeros(chunk)
        else:
            noise = np.random.normal(scale=stddev, size=chunk)
        # else: noise = np.random.uniform(high=stddev, size=chunk)
        if ra is None:
            return noise
        elif ra2 is None:
            return mult_ra * (ra + noise)
        else:
            return (mult_ra * ra) + (mult_ra2 * ra2) + noise

    def ccol(flip_chance=0, ra=None, ra2=None, ra_thr=0, reverse_cats=False):
        if ra is None:
            ra = np.random.random(chunk)
            c = ra < ra_thr
        elif ra2 is None:
            if np.issubdtype(ra.dtype, np.number):
                c = ra < ra_thr
            else:
                c = ra
        else:
            c = np.logical_xor(ra, ra2)

        flip = np.random.random(chunk) < flip_chance
        c = np.logical_xor(c, flip)
        if reverse_cats:
            c = ~c
        return c

    # n0 = ncol()
    # n1 = ncol()
    # n2 = ncol(0, n1, mult_ra=2)
    # n3 = ncol()
    # n4 = ncol(0, n1, n3, .5, .5)
    # c5 = ccol(ra_thr=.5)
    # c6 = ccol(ra_thr=.5)
    # c7 = ccol(0, c6)
    # c8 = ccol(0, n3, ra_thr=0)
    # c9 = ccol(0, c6, c8)
    # t = tfunc(c7, c7, [n2], [n2], [n2], [n2], [1], [1], [1], [1],
    #     [.5, 0, 0, .5], [False, False, False, True]) #c7:n2<.5, ~c7:n2>.5
    # tenbatch = np.concatenate([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t], axis=1)
    # dataset = np.concatenate([dataset, tenbatch])
    # pd.DataFrame(dataset).to_csv('bla.csv', sep='\t')
    # print(dataset)
    # #print('0',n0,'1',n1,'2',n2,'3',n3,'4',n4,'5',c5,'6',c6,'7',c7,'8',c8,'9',c9,'C',t)
    # raise

    # 0..999
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 1000..1999 inc num f
    n0 = ncol() + lin_incr(0, 2)  # to 2
    n1 = ncol() + lin_incr(0, 1)  # to 1
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol() + lin_incr(0, -1)  # to -1
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 2000..2999 inc cat f
    n0 = ncol() + 2
    n1 = ncol() + 1
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol() - 1
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=lin_incr(0.5, 0.8))  # to .8
    c6 = ccol(ra_thr=lin_incr(0.5, 0.2))  # to .2
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=lin_incr(0, -1))  # to -1
    c9 = ccol(err, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 3000..3999 inc coef f
    n0 = ncol() + 2
    n1 = ncol() + 1
    n2 = ncol(err, n1, mult_ra=lin_incr(2, 5))  # to 5
    n3 = ncol() - 1
    n4 = ncol(err, n1, n3, lin_incr(0.5, 0.8), lin_incr(0.5, 0.2))  # to (.8, .2)
    c5 = ccol(ra_thr=0.8)
    c6 = ccol(ra_thr=0.2)
    c7 = ccol(lin_incr(err, 0.25), c6)  # to .25
    c8 = ccol(err, n3, ra_thr=-1)
    c9 = ccol(lin_incr(err, 0.25), c6, c8)  # to .25
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 4000..4999 gra num f
    n0 = ncol() + gradual(2, -2)  # to -2
    n1 = ncol() + gradual(1, -1)  # to -1
    n2 = ncol(err, n1, mult_ra=5)
    n3 = ncol() + gradual(-1, 1)  # to 1
    n4 = ncol(err, n1, n3, 0.8, 0.2)
    c5 = ccol(ra_thr=0.8)
    c6 = ccol(ra_thr=0.2)
    c7 = ccol(0.25, c6)
    c8 = ccol(err, n3, ra_thr=-1)
    c9 = ccol(0.25, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 5000..5999 gra cat f
    n0 = ncol() - 2
    n1 = ncol() - 1
    n2 = ncol(err, n1, mult_ra=5)
    n3 = ncol() + 1
    n4 = ncol(err, n1, n3, 0.8, 0.2)
    c5 = ccol(ra_thr=gradual(0.8, 0.2))  # to .2
    c6 = ccol(ra_thr=gradual(0.2, 0.8))  # to .8
    c7 = ccol(0.25, c6)
    c8 = ccol(err, n3, ra_thr=gradual(-1, 1))  # to 1
    c9 = ccol(0.25, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 6000..6999 gra coef f
    n0 = ncol() - 2
    n1 = ncol() - 1
    n2 = ncol(err, n1, mult_ra=gradual(5, -3))  # to -3
    n3 = ncol() + 1
    n4 = ncol(err, n1, n3, gradual(0.8, 0.2), gradual(0.2, 0.8))  # to (.2, .8)
    c5 = ccol(ra_thr=0.2)
    c6 = ccol(ra_thr=0.8)
    c7 = ccol(gradual(0.25, 0.375), c6)  # to .375
    c8 = ccol(err, n3, ra_thr=1)
    c9 = ccol(0.25, gradual(c6, c5), c8)  # to c5
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 7000..7999 sudden num f
    n0 = ncol()  # to 0
    n1 = ncol()  # to 0
    n2 = ncol(err, n1, mult_ra=-3)
    n3 = ncol()  # to 0
    n4 = ncol(err, n1, n3, 0.2, 0.8)
    c5 = ccol(ra_thr=0.2)
    c6 = ccol(ra_thr=0.8)
    c7 = ccol(0.375, c6)
    c8 = ccol(err, n3, ra_thr=1)
    c9 = ccol(0.25, c5, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 8000..8999 sudden num f
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=-3)
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.2, 0.8)
    c5 = ccol(ra_thr=0.5)  # to .5
    c6 = ccol(ra_thr=0.5)  # to .5
    c7 = ccol(0.375, c6)
    c8 = ccol(err, n3, ra_thr=0)  # to 0
    c9 = ccol(0.25, c5, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 9000..9999 sudden coef f
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=2)  # to 2
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.5, 0.5)  # to (.5, .5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)  # to .125
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)  # to .125, c6
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0.5, 0, 0, 0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 10000..10999 incremental label
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)
    t = tfunc(
        c7,
        c7,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [lin_incr(0.5, -0.5), 0, 0, lin_incr(0.5, -0.5)],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 11000..11999 gradual label
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)
    grad = gradual(c7, c6)
    t = tfunc(
        grad,
        grad,
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [n2, n3],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [-0.5, 0, 0, -0.5],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    # 12000..12999 sudden label
    n0 = ncol()
    n1 = ncol()
    n2 = ncol(err, n1, mult_ra=2)
    n3 = ncol()
    n4 = ncol(err, n1, n3, 0.5, 0.5)
    c5 = ccol(ra_thr=0.5)
    c6 = ccol(ra_thr=0.5)
    c7 = ccol(err, c6)
    c8 = ccol(err, n3, ra_thr=0)
    c9 = ccol(err, c6, c8)
    t = tfunc(
        c5,
        c5,
        [n0, n4],
        [n0, n4],
        [n0, n4],
        [n0, n4],
        [2, 1],
        [2, 1],
        [2, 1],
        [2, 1],
        [1, 0, 0, 1],
        [False, False, False, True],
        err,
    )  # c7:n2<.5, ~c7:n2>.5
    tenbatch = np.array([n0, n1, n2, n3, n4, c5, c6, c7, c8, c9, t]).T
    dataset = np.concatenate([dataset, tenbatch])

    num_cols = ["n0", "n1", "n2", "n3", "n4"]
    cat_cols = ["c5", "c6", "c7", "c8", "c9"]
    label = ["class"]
    df_dataset = pd.DataFrame(dataset, columns=num_cols + cat_cols + label)
    df_dataset[cat_cols] = df_dataset[cat_cols].replace({0: "a", 1: "b"})
    df_dataset[label] = df_dataset[label].replace({0: "A", 1: "B"})
    df_dataset.to_csv(f"{filename}.csv", index=False)
    df_dataset.to_pickle(filename)

In [8]:
def main():

    error_dists = ["normal", "uniform"]
    corr_funcs = ["sum", "mean", "min", "max"]

    parser = argparse.ArgumentParser(
        description="Generates a dataset of a correlated feature and noise"
    )
    parser.add_argument(
        "-d", required=True, type=int, help="The no. of instances of the data set"
    )
    parser.add_argument("-f", required=True, type=int, help="The no. of noise features")
    parser.add_argument(
        "-c", required=True, type=int, help="The no. of correlating features"
    )
    parser.add_argument(
        "-r", required=True, type=float, help="The ± range of the random component"
    )
    parser.add_argument(
        "--error",
        required=False,
        choices=error_dists,
        default=error_dists[0],
        help="The type of error",
    )
    parser.add_argument(
        "--cfunc",
        required=False,
        choices=corr_funcs,
        default=corr_funcs[0],
        help="The function that fuses the correlated features",
    )
    parser.add_argument(
        "--coffset",
        required=False,
        type=float,
        default=0.0,
        help="A constant offset for correlated features",
    )
    parser.add_argument(
        "--ccoeff",
        required=False,
        type=float,
        default=2.0,
        help="A factor for correlated features",
    )
    parser.add_argument(
        "--cexpo",
        required=False,
        type=float,
        default=1.0,
        help="An exponential for correlated features",
    )
    parser.add_argument(
        "--cquant",
        required=False,
        type=int,
        default=0,
        help="If set will quantize the correlated feature by multiplying, flooring and dividing with this value",
    )
    parser.add_argument(
        "--filepath",
        required=False,
        default=None,
        help="Specify to set a custom save name",
    )
    args = parser.parse_args()

    if args.f < 0:
        raise ValueError(f"Negative feature values are prohibited (f = {args.f})")
    if args.c < 0:
        raise ValueError(f"Negative feature values are prohibited (c = {args.c})")
    if args.c + args.f == 0:
        raise ValueError("At least one feature has to be generated")

    folder = (
        os.path.join("consts.DIR_CSV", "corr", f"f{args.f}_c{args.c}_r{args.r}")
        if args.filepath is None
        else os.path.dirname(args.filepath)
    )
    filename = (
        os.path.join(folder, "raw_data.pkl.gzip")
        if args.filepath is None
        else args.filepath
    )
    if not os.path.exists(folder):
        os.mkdir(folder)

    noise = np.random.random([args.d, args.f])
    corrs = np.random.random([args.d, args.c])

    if args.cfunc == corr_funcs[0]:
        fcorr = corrs.sum(axis=1, keepdims=True)
    elif args.cfunc == corr_funcs[1]:
        fcorr = corrs.mean(axis=1, keepdims=True)
    elif args.cfunc == corr_funcs[2]:
        fcorr = corrs.min(axis=1, keepdims=True)
    elif args.cfunc == corr_funcs[3]:
        fcorr = corrs.max(axis=1, keepdims=True)

    c_true = fcorr**args.cexpo * args.ccoeff + args.coffset

    if args.error == error_dists[0]:
        c = c_true + np.random.normal(0, args.r, [args.d, 1])
    elif args.error == error_dists[1]:
        c = c_true + np.random.uniform(-args.r, args.r, [args.d, 1])

    if args.cquant != 0:
        c = np.floor(c * args.cquant) / args.cquant

    label = (c > c_true).reshape([args.d, 1])

    cols = (
        [f"N{x}" for x in range(args.f)] + [f"C{x}" for x in range(args.c)] + ["T", "L"]
    )
    data = np.concatenate((noise, corrs, c, label), axis=1)

    df = pd.DataFrame(data, columns=cols)
    df.to_pickle(filename)

In [9]:
if __name__ == "__main__":
    # filepath = os.path.join("consts.DIR_CSV", "cfpdss//raw_data.pkl.gzip")
    # change_label_func_splits(filepath)
    # df = change_label_func(filepath)
    # cfpdss_dataset()
    # gen_mixed_dataset()
    # main()
    []